In [1]:
%pip install -q -U anthropic
%pip install -q -U lightning
%pip install -q -U bitsandbytes
%pip install -q -U transformers
%pip install -q -U peft
%pip install -q -U accelerate
%pip install -q -U wandb
%pip install -q -U datasets
%pip install -q -U prodigyopt
%pip install pdf2image
!apt-get update --yes
!apt-get install -y poppler-utils

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Get:1 file:/var/cuda-repo-ubuntu2204-12-0-local  InRelease [1575 B]
Get:1 file:/var/cuda-repo-ubuntu2204-12-0-local  InRelease [1575 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://deb.nodesource.com/node_20.x nodistro InRelease                  
Hit:4 http://archive.ubuntu.com/

In [2]:
import torch
import os
import json
import random
from transformers import AutoProcessor, VisionEncoderDecoderModel
from torch.utils.data import Dataset, DataLoader
import torchvision as tv
from transformers import DonutProcessor
from torch.utils.data import DataLoader
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
import os, torch, torch.nn as nn, torch.utils.data as data, torchvision as tv
import lightning as L
from transformers import DonutProcessor, VisionEncoderDecoderModel
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
import wandb
import os
from IPython.display import Image
from tqdm.auto import tqdm
from IPython.display import Image
import pickle
import torchvision.transforms as T
from datasets import Dataset
from datasets import IterableDataset
from tqdm.auto import tqdm
import unicodedata
import os
import gc
from torch.utils.data import Dataset
import torchvision as tv
import torchvision.transforms as T
from transformers import DonutProcessor
import random
from torch.utils.data import Subset
from peft import LoraConfig
from torch.optim.lr_scheduler import LinearLR
import lightning as L
from prodigyopt import Prodigy
from datetime import datetime
import torch.multiprocessing as mp
from torch.cuda.amp import autocast, GradScaler
from pdf2image import convert_from_path, convert_from_bytes
from transformers import VisionEncoderDecoderModel, DonutProcessor
from pytorch_lightning.loggers import WandbLogger
from pdf2image import convert_from_path
from torchvision.transforms import ToTensor

2024-09-25 10:10:57.867463: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 10:10:57.867507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 10:10:57.868625: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 10:10:57.875536: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-25 10:10:58.838601: W tensorflow/compiler/tf2

In [3]:
DATASET_PATH = "./documents"
PDF_PATH = os.path.join(DATASET_PATH, "german_pdf_files")
IMAGE_PATH = os.path.join(DATASET_PATH, "german_img_files")


class DocMeta:
    def __init__(self, dataset_path=DATASET_PATH, image_path=IMAGE_PATH):
        self.dataset_path = dataset_path
        self.image_path = image_path
        self.extraction = self.load_extraction(self.dataset_path)
        self.mapping = self.load_mapping(self.image_path)

    def get_random_urls(self, num=100):
        return random.sample(list(self.extraction.keys()), num)

    def load_extraction(self, path):
        with open(os.path.join(path, "extraction.json"), "r") as f:
            return json.load(f)

    def load_mapping(self, path):
        with open(os.path.join(path, "mapping.json"), "r") as f:
            return json.load(f)

    def get_image_path(self, url):
        return os.path.join(self.image_path, self.mapping[url])

    def get_image(self, url):
        img_path = self.get_image_path(url)
        return Image(filename=img_path)


doc_meta = DocMeta()

In [4]:
class DocMetaDataset(Dataset):
    def __init__(self, doc_meta, output_dir, ocr=False):
        self.doc_meta = doc_meta
        self.task_start_token = "<s>"
        self.eos_token = "</s>"
        self.processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
        self.processor.tokenizer.add_special_tokens(
            {"additional_special_tokens": [self.task_start_token, self.eos_token]}
        )
        self.processor.feature_extractor.size = [2160, 3840]
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        self.data = self._prepare_data()
        self.ocr = ocr

    def _prepare_data(self):
        data = []
        for url in tqdm(
            list(self.doc_meta.extraction.keys()), desc="Preparing dataset"
        ):
            main_author = self.doc_meta.extraction[url]["metadata"]["main_author"]

            transcript = self.doc_meta.extraction[url]["transcript"]

            transcript = self.task_start_token + transcript + self.eos_token

            if not main_author is None:

                if "name" in main_author:

                    author = main_author["name"]

                elif "first_name" in main_author and "last_name" in main_author:

                    author = ""
                    if main_author["first_name"] is not None:
                        author += main_author["first_name"]
                    if main_author["last_name"] is not None:
                        author += " " + main_author["last_name"]

                elif "department" in main_author:

                    author = main_author["department"]

                else:
                    continue
            else:
                continue

            author = self.task_start_token + author + self.eos_token
            image_path = self.doc_meta.get_image_path(url)

            image_data = tv.io.read_image(unicodedata.normalize("NFC", image_path))
            res = tv.transforms.functional.get_image_size(image_data)

            if res[0] > 2160 or res[1] > 3840:
                continue

            if author.replace(" ", "") == "<s></s>":
                continue

            if len(author) > 80:
                continue

            data.append((image_path, author, url, transcript))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, author, url, transcript = self.data[idx]

        image_data = tv.io.read_image(unicodedata.normalize("NFC", image_path))
        image_data = image_data.permute(1, 2, 0)

        pixel_values = self.processor(image_data, return_tensors="pt").pixel_values
        pixel_values = pixel_values.squeeze()

        if not self.ocr:
            input_ids_author = (
                self.processor.tokenizer(
                    author,
                    add_special_tokens=False,
                    padding="max_length",
                    truncation=True,
                    max_length=80,
                    return_tensors="pt",
                )["input_ids"]
                .squeeze(0)
                .long()
            )

            author_target = input_ids_author.clone()
            author_target[author_target == self.processor.tokenizer.pad_token_id] = -100
            return {
                "pixel": pixel_values,
                "target": author_target,
                "url": url,
            }

        if self.ocr:
            input_ids_transcript = (
                self.processor.tokenizer(
                    transcript,
                    add_special_tokens=False,
                    padding="max_length",
                    truncation=True,
                    max_length=300,
                    return_tensors="pt",
                )["input_ids"]
                .squeeze(0)
                .long()
            )
            transcript_target = input_ids_transcript.clone()
            transcript_target[
                transcript_target == self.processor.tokenizer.pad_token_id
            ] = -100
            return {
                "pixel": pixel_values,
                "target": transcript_target,
                "url": url,
            }

    def get_split(self, train_ratio=0.8, val_ratio=0.1, seed=None):
        total_size = len(self.data)
        train_size = int(train_ratio * total_size)
        val_size = int(val_ratio * total_size)
        test_size = total_size - train_size - val_size  # Ensure all data is used

        indices = list(range(total_size))
        if seed is not None:
            random.Random(seed).shuffle(indices)
        else:
            random.shuffle(indices)

        train_indices = indices[:train_size]
        val_indices = indices[train_size : train_size + val_size]
        test_indices = indices[train_size + val_size :]

        train_dataset = Subset(self, train_indices)
        val_dataset = Subset(self, val_indices)
        test_dataset = Subset(self, test_indices)

        return train_dataset, val_dataset, test_dataset

In [5]:
dataset = DocMetaDataset(doc_meta, "./")

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/donut/processing_donut.py:193: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(


Preparing dataset:   0%|          | 0/4702 [00:00<?, ?it/s]

In [6]:
train, val, test = dataset.get_split(0.8, 0.18)
len(train), len(val), len(test)

(2892, 650, 74)

In [7]:
dataset.ocr = False

In [8]:
batch_size = 1
num_workers = 0

train_dataloader = DataLoader(
    train, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True
)

val_dataloader = DataLoader(
    val, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True
)

test_dataloader = DataLoader(
    test, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True
)

In [9]:
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base")

model.decoder.resize_token_embeddings(len(dataset.processor.tokenizer))
model.config.pad_token_id = dataset.processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = dataset.processor.tokenizer.convert_tokens_to_ids(
    ["<s>"]
)[0]


# peft_config = LoraConfig(
#    lora_alpha=128, lora_dropout=0.05, r=256, bias="none", target_modules="all-linear"
# )
#
# model = get_peft_model(model, peft_config)

model = model.to("cuda")

# model.print_trainable_parameters()

/usr/local/lib/python3.11/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
torch.set_float32_matmul_precision("medium")

In [11]:
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jonasg. Use `wandb login --relogin` to force relogin


True

In [12]:
# compiled_model = torch.compile(model) # not working

In [13]:
scaler = GradScaler()  # for scaling the loss


optimizer = Prodigy(model.parameters(), lr=1.0)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=10 * len(train_dataloader)
)


run = wandb.init(
    # Set the project where this run will be logged
    project="donut_torch",
)


def train_one_epoch(epoch_index):
    running_loss = 0.0
    last_loss = 0.0

    for i, data in enumerate(train_dataloader):
        batch = data

        optimizer.zero_grad()

        with autocast():

            outputs = model(
                pixel_values=batch["pixel"].to("cuda"),
                labels=batch["target"].to("cuda"),
            )

            loss = outputs.loss

        scaler.scale(loss).backward()  # why?
        scaler.step(optimizer)  # why?
        scaler.update()  # why?

        running_loss += loss.item()
        if i % 10 == 9:
            last_loss = running_loss / 10  # loss per batch
            print("  batch {} loss: {}".format(i + 1, last_loss))
            wandb.log({"train_loss": last_loss})
            running_loss = 0.0

        # Test loss
        if i % 200 == 190:
            running_vloss = 0
            with torch.no_grad():
                for i, vdata in enumerate(test_dataloader):
                    vbatch = vdata
                    with autocast():
                        voutputs = model(
                            pixel_values=vbatch["pixel"].to("cuda"),
                            labels=vbatch["target"].to("cuda"),
                        )
                        vloss = voutputs.loss
                    running_vloss += vloss
            avg_vloss = running_vloss / (i + 1)
            print("  vloss: {}".format(avg_vloss))
            wandb.log({"v_loss": avg_vloss})

    scheduler.step()

    return last_loss

In [14]:
# TODO: epochs too long

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.0


for epoch in range(EPOCHS):
    print("EPOCH {}:".format(epoch_number + 1))

    model.train(True)
    avg_loss = train_one_epoch(epoch_number)

    running_vloss = 0.0

    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(val_dataloader):
            vbatch = vdata
            with autocast():
                voutputs = model(
                    pixel_values=vbatch["pixel"].to("cuda"),
                    labels=vbatch["target"].to("cuda"),
                )
                vloss = voutputs.loss
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print("LOSS train {} valid {}".format(avg_loss, avg_vloss))
    wandb.log({"val_loss": avg_vloss})

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        print("Saving model ...")
        best_vloss = avg_vloss
        model_path = "model_{}_{}".format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)
        wandb.save(model_path)

    epoch_number += 1

EPOCH 1:
  batch 10 loss: 8.758977508544922
  batch 20 loss: 6.159610867500305
  batch 30 loss: 6.076745796203613
  batch 40 loss: 5.111327195167542
  batch 50 loss: 4.064751261472702
  batch 60 loss: 3.556390333175659
  batch 70 loss: 4.1105811357498165
  batch 80 loss: 3.5604020714759828
  batch 90 loss: 3.8712538480758667
  batch 100 loss: 3.5521793365478516
  batch 110 loss: 3.576555597782135
  batch 120 loss: 3.102889132499695
  batch 130 loss: 2.9446736335754395
  batch 140 loss: 2.813510650396347
  batch 150 loss: 2.658648061752319
  batch 160 loss: 3.615069258213043
  batch 170 loss: 3.094812422990799
  batch 180 loss: 2.334120738506317
  batch 190 loss: 2.8506755948066713
  vloss: 2.6061649322509766
  batch 200 loss: 2.973324012756348
  batch 210 loss: 3.273329347372055
  batch 220 loss: 1.9528379440307617
  batch 230 loss: 2.2676897823810576
  batch 240 loss: 3.1206485271453857
  batch 250 loss: 2.6778649628162383
  batch 260 loss: 3.4383625209331514
  batch 270 loss: 3.78109

# Storage and Quantization

In [20]:
from huggingface_hub import login

login()

In [29]:
# does not work when using peft LoRA
model.push_to_hub("doc_meta")

[2024-09-25 10:02:11,206] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


model.safetensors:   0%|          | 0.00/809M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sodowo/doc_meta/commit/687033bdb1eb714953d4973675c1b77864674a14', commit_message='Upload model', commit_description='', oid='687033bdb1eb714953d4973675c1b77864674a14', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sodowo/doc_meta', endpoint='https://huggingface.co', repo_type='model', repo_id='sodowo/doc_meta'), pr_revision=None, pr_num=None)

# Storage and inference

In [34]:
from transformers import AutoTokenizer, AutoModel

# model = AutoModel.from_pretrained("sodowo/doc_meta")

model = VisionEncoderDecoderModel.from_pretrained("sodowo/doc_meta")

model.decoder.resize_token_embeddings(len(dataset.processor.tokenizer))
model.config.pad_token_id = dataset.processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = dataset.processor.tokenizer.convert_tokens_to_ids(
    ["<s>"]
)[0]


peft_config = LoraConfig(
    lora_alpha=128, lora_dropout=0.05, r=256, bias="none", target_modules="all-linear"
)

model = get_peft_model(model, peft_config)


model.load_state_dict(torch.load("model_final.pt"))

model.safetensors:   0%|          | 0.00/809M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

<All keys matched successfully>

In [36]:
eval_model = model.eval()

In [37]:
image_data = convert_from_path("./uni_hd.pdf", fmt="jpeg", first_page=1, last_page=1)[0]

In [38]:
transform = ToTensor()
image_data = transform(image_data)
image_data = image_data.permute(1, 2, 0)

In [39]:
pixel_values = dataset.processor(image_data, return_tensors="pt").pixel_values
pixel_values = pixel_values.squeeze()

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


In [42]:
model.to("cpu")
with torch.no_grad():
    generated_ids = eval_model.generate(pixel_values.unsqueeze(0).to("cpu"))

In [41]:
generated_ids

tensor([[0, 2]], device='cuda:0')

In [49]:
predicted_label = dataset.processor.batch_decode(
    generated_ids, skip_special_tokens=True
)[0]

In [50]:
predicted_label

''